## $\color{Green}{\text{Check Individual Module (AR5K14) Results}}$

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import netCDF4 as nc
from pandas.plotting import table 
import xarray as xr

#### $\color{blue}{\text{List of Modules and Data to be used}}$

In [2]:
# Load FACTS workflow data as a dataframe.
# df = pd.read_fwf('../001_GMD_IP_Modules_Data/ModuleTest_DataFile_List_ar5k14.txt',comment = '#')
df = pd.read_fwf('../001_GMD_IP_Modules_Data/ModuleTest_DataFile_List_ar5k14-221217.txt',comment = '#')
df

,Component,Module,sub_Module,DataFile
0,TE,kopp14,sterodynamics,k14sterodynamics.kopp14.sterodynamics_globalsl.nc
1,TE,ipccar5,thermalexpansion,ar5TE.ipccar5.thermalexpansion_globalsl.nc
2,Glaciers,kopp14,glaciers,k14glaciers.kopp14.glaciers_globalsl.nc
3,Glaciers,ipccar5,glaciers,ar5glaciers.ipccar5.glaciers_globalsl.nc
4,GrIS,ipccar5,icesheets,ar5icesheets.ipccar5.icesheets_GIS_globalsl.nc
5,GrIS,kopp14,icesheets,k14icesheets.kopp14.icesheets_GIS_globalsl.nc
6,AIS,ipccar5,icesheets,ar5icesheets.ipccar5.icesheets_AIS_globalsl.nc
7,AIS,kopp14,AIS,k14icesheets.kopp14.icesheets_AIS_globalsl.nc
8,EAIS,ipccar5,icesheets,ar5icesheets.ipccar5.icesheets_EAIS_globalsl.nc
9,EAIS,kopp14,icesheets,k14icesheets.kopp14.icesheets_EAIS_globalsl.nc


#### $\color{blue}{\text{Write nc data details to a file}}$

In [3]:
EXPDATE     = 221217
MODNO       = df.index.values;   # Number of datasets Module outputs
SCENARIO    = ['rcp26', 'rcp45', 'rcp85']; 
#        
file = open("ModuleData_Info_AR5K14-{arg1}.txt".format(arg1=EXPDATE), "w")
#
for val in MODNO: 
    for scenario in SCENARIO:
        #
        MODULE = df["Module"][val];  SUB_MODULE = df["sub_Module"][val]; COMP = df["Component"][val];
        # Skip if data is absent.
        if df["DataFile"][val] == 'XXX':
           continue 
        #
        # Pick the data files & Import nc file to dataframe. 
        dataFOLDER  = '/projects/kopp/facts-experiments/{arg2}/ar5k14.{arg1}/output/'.format(arg1=scenario,arg2=EXPDATE)
        dataFILE    = 'ar5k14.{arg1}.'.format(arg1=scenario) + df["DataFile"][val]
        d_nc        = xr.open_dataset(dataFOLDER + dataFILE)
        d_nc
    file.write( "DataFILE ==> "+str(df["DataFile"][val])+"\n"+ str(d_nc.dims) + "\n" + str(d_nc.data_vars)+ "\n" + str(d_nc.coords)+"\n"+"-------------"+"\n"+"\n" )
        #
file.close()


#### $\color{Red}{\text{Load Module data from ==> facts-experiments}}$

In [4]:
EXPDATE     = 221217
MODNO       = df.index.values;   # Number of datasets Module outputs
# MODNO       = np.arange(7);
SCENARIO    = ['rcp26', 'rcp45', 'rcp85']; 
YEAR        = 2100; # Percentile Year for the table.    
DATAEXCEPT = ['ar5TE.ipccar5.thermalexpansion_globalsl.nc']
#
# Temp DF to store all .nc O/P.      
df_TEMP      = []
#        
for scenario in SCENARIO:
    # Define pd DF columns to append.
    PERCENTILE  = '%ile({arg2})'.format(arg2=scenario)
    percentile = np.nan
    df[PERCENTILE] = percentile
    df[PERCENTILE] = df[PERCENTILE].astype('object') 
    #
    for val in MODNO: 
        #
        MODULE = df["Module"][val];  SUB_MODULE = df["sub_Module"][val]; COMP = df["Component"][val]; DATAFILE = df["DataFile"][val];
        # Skip if data is absent.
        if df["DataFile"][val] == 'XXX':
           continue 
        #
        # Pick the data files & Import nc file to dataframe. 
        dataFOLDER  = '/projects/kopp/facts-experiments/{arg2}/ar5k14.{arg1}/output/'.format(arg1=scenario,arg2=EXPDATE)
        dataFILE    = 'ar5k14.{arg1}.'.format(arg1=scenario) + df["DataFile"][val]
        d_nc        = xr.open_dataset(dataFOLDER + dataFILE)
        d_nc
        #
        # Percentile calculation.
        percentList = [50, 17, 83]          #[50, 5, 17, 83, 95]
        # Find year index to pick SLC value
        if any(x in DATAFILE for x in DATAEXCEPT):
            Yind        = np.where(d_nc["year"].values == YEAR)[0][0]
        else: 
            Yind        = np.where(d_nc["years"].values == YEAR)[0][0]
        
        # 
        if SUB_MODULE == 'temperature':
            if  DATAFILE == 'temperature.fair.temperature_oceantemp.nc':
                    GMSL    = (d_nc["deep_ocean_temperature"][:,Yind,0].values)/1   
            elif DATAFILE == 'temperature.fair.temperature_ohc.nc':
                    GMSL    = (d_nc["ocean_heat_content"][:,Yind,0].values)/1   
            else:
                GMSL    = (d_nc["surface_temperature"][:,Yind,0].values)/1 
                #
        else:
            GMSL    = (d_nc["sea_level_change"][:,Yind,0].values)/1       
        # Find Percentiles.
        pcntle = np.percentile(GMSL[:], percentList );    pcntle = np.around(pcntle,2)
        #
        # df.at[val,PERCENTILE] = [pcntle[0], pcntle[1],pcntle[2]]
        df.at[val,PERCENTILE] = f'{pcntle[0]:2.2f} ({pcntle[1]:2.2f} , {pcntle[2]:2.2f})' 
        # Print the OP.
print('GMSL Component Projections for ',d_nc["years"][Yind].values)    
#show(df)
df

GMSL Component Projections for  2100


,Component,Module,sub_Module,DataFile,%ile(rcp26),%ile(rcp45),%ile(rcp85)
0,TE,kopp14,sterodynamics,k14sterodynamics.kopp14.sterodynamics_globalsl.nc,"162.27 (102.51 , 222.04)","228.46 (152.46 , 304.45)","338.43 (257.53 , 419.33)"
1,TE,ipccar5,thermalexpansion,ar5TE.ipccar5.thermalexpansion_globalsl.nc,"153.14 (127.88 , 179.03)","205.14 (177.48 , 233.50)","318.85 (279.75 , 358.91)"
2,Glaciers,kopp14,glaciers,k14glaciers.kopp14.glaciers_globalsl.nc,"111.88 (83.61 , 139.95)","127.26 (94.23 , 159.97)","169.56 (131.74 , 205.63)"
3,Glaciers,ipccar5,glaciers,ar5glaciers.ipccar5.glaciers_globalsl.nc,"94.00 (61.00 , 130.17)","117.00 (80.00 , 161.00)","161.00 (112.00 , 221.00)"
4,GrIS,ipccar5,icesheets,ar5icesheets.ipccar5.icesheets_GIS_globalsl.nc,"76.00 (54.00 , 102.00)","93.00 (67.00 , 127.00)","160.50 (113.00 , 223.00)"
5,GrIS,kopp14,icesheets,k14icesheets.kopp14.icesheets_GIS_globalsl.nc,"59.00 (34.00 , 109.00)","84.00 (34.00 , 147.00)","135.00 (72.00 , 247.00)"
6,AIS,ipccar5,icesheets,ar5icesheets.ipccar5.icesheets_AIS_globalsl.nc,"63.50 (-12.00 , 137.00)","54.50 (-21.00 , 130.00)","38.00 (-39.00 , 115.17)"
7,AIS,kopp14,AIS,k14icesheets.kopp14.icesheets_AIS_globalsl.nc,"59.00 (-46.17 , 165.00)","54.00 (-57.00 , 155.00)","37.50 (-81.00 , 144.17)"
8,EAIS,ipccar5,icesheets,ar5icesheets.ipccar5.icesheets_EAIS_globalsl.nc,"9.00 (-1.00 , 19.00)","7.00 (-3.00 , 18.00)","5.00 (-5.00 , 16.00)"
9,EAIS,kopp14,icesheets,k14icesheets.kopp14.icesheets_EAIS_globalsl.nc,"-6.00 (-63.00 , 84.17)","-8.00 (-69.00 , 83.00)","-16.00 (-81.17 , 80.00)"


In [ ]:
df.to_excel(r'Table_CheckModule_AR5K14_GMSL-Proj_2100-{arg1}.xlsx'.format(arg1=EXPDATE), index=False)
